# Práctica: Limpieza de Datos del Titanic

## 1. Carga de Datos Reales (Desde URL)
Usaremos el dataset oficial del Titanic alojado en GitHub.
Para cumplir con el requisito de practicar "Duplicados", simularemos un error de carga duplicando intencionalmente las primeras 5 filas (algo muy común en bases de datos reales).

In [4]:
import pandas as pd
import numpy as np

# URL directa al archivo CSV real (Raw)
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"

# Leemos el archivo real
df = pd.read_csv(url)

# SIMULACIÓN DE ERROR: Duplicamos las primeras 5 filas para tener qué limpiar
# (Imagina que el sistema cargó dos veces los primeros registros)
df = pd.concat([df, df.head(5)], ignore_index=True)

print("--- Dataset Titanic Cargado ---")
print(f"Dimensiones: {df.shape}")
display(df.head())

--- Dataset Titanic Cargado ---
Dimensiones: (896, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 2. Diagnóstico de Salud de los Datos
Analizamos cuántos registros están repetidos y qué columnas tienen información faltante (`NaN`).

In [ ]:
# 1. Detección de Duplicados
duplicados = df.duplicated().sum()
print(f"⚠️ Filas duplicadas detectadas: {duplicados}")

# 2. Detección de Nulos
print("\n⚠️ Conteo de Valores Nulos por columna:")
# Filtramos solo las columnas que tienen nulos para ver mejor
nulos = df.isnull().sum()
print(nulos[nulos > 0])

## 3. Eliminación de Duplicados
Primero eliminamos las filas repetidas para no sesgar los cálculos estadísticos posteriores.

In [ ]:
# Eliminamos duplicados
df_limpio = df.drop_duplicates().copy()

print(f"Filas antes: {len(df)}")
print(f"Filas después: {len(df_limpio)}")

# Verificación
assert df_limpio.duplicated().sum() == 0, "Error: Aún quedan duplicados"

## 4. Tratamiento de Valores Nulos
Tenemos dos casos reales:
1.  **Age (Edad):** Es un valor numérico. Usaremos el **Promedio (Mean)** para rellenar los huecos, asumiendo que los pasajeros sin edad tenían la edad promedio del barco.
2.  **Cabin (Cabina):** Es texto y faltan muchos datos. Usaremos una constante **"Desconocido"**.

In [ ]:
# ESTRATEGIA 1: Numérica (Promedio) para 'Age'
promedio_edad = df_limpio['Age'].mean()
print(f"🎂 Edad promedio calculada: {promedio_edad:.1f} años")

df_limpio['Age'] = df_limpio['Age'].fillna(promedio_edad)


# ESTRATEGIA 2: Categórica (Valor Fijo) para 'Cabin'
df_limpio['Cabin'] = df_limpio['Cabin'].fillna("Desconocido")

# Verificamos visualmente los cambios
print("\n--- Verificación de Imputación ---")
# Mostramos filas que antes tenían nulos para ver cómo quedaron
display(df_limpio[['Name', 'Age', 'Cabin']].sample(5))

## 📌 Validación y Exportación
Confirmamos que el dataset está limpio (0 nulos críticos, 0 duplicados) y listo para usarse.

In [ ]:
# Revisión final
print("Nulos restantes en Edad:", df_limpio['Age'].isnull().sum())
print("Nulos restantes en Cabina:", df_limpio['Cabin'].isnull().sum())
print("Duplicados restantes:", df_limpio.duplicated().sum())

print("\n✅ El dataset del Titanic ha sido limpiado exitosamente.")